## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config_copy import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("..\Weather_Database\WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,74.86,80,100,18.63,light rain
1,1,Kapaa,US,22.0752,-159.3190,80.58,76,1,3.00,clear sky
2,2,Portland,US,45.5234,-122.6762,57.36,86,90,3.00,light rain
3,3,Port Alfred,ZA,-33.5906,26.8910,66.56,82,85,29.17,overcast clouds
4,4,Bambous Virieux,MU,-20.3428,57.7575,74.21,73,20,14.79,few clouds


In [10]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [11]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Kapaa,US,22.0752,-159.3190,80.58,76,1,3.00,clear sky
11,11,Piacabucu,BR,-10.4056,-36.4344,78.55,78,37,11.07,scattered clouds
13,13,Faanui,PF,-16.4833,-151.7500,78.24,77,66,8.43,broken clouds
16,16,Hilo,US,19.7297,-155.0900,80.26,67,1,4.61,clear sky
18,18,Hambantota,LK,6.1241,81.1185,76.55,87,100,6.38,overcast clouds
27,27,Aranos,NaN,-24.1333,19.1167,78.96,9,20,9.24,few clouds
30,30,Bengkulu,ID,-3.8004,102.2655,75.67,89,100,5.97,moderate rain
31,31,Yenagoa,NG,4.9247,6.2642,76.73,88,35,4.07,scattered clouds
33,33,Lazaro Cardenas,MX,17.9583,-102.2000,84.74,71,93,4.56,overcast clouds
42,42,Borba,BR,-4.3878,-59.5939,85.19,75,99,3.06,overcast clouds


In [12]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                4
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [13]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [14]:
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [15]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Kapaa,US,80.58,clear sky,22.0752,-159.3190,
11,Piacabucu,BR,78.55,scattered clouds,-10.4056,-36.4344,
13,Faanui,PF,78.24,broken clouds,-16.4833,-151.7500,
16,Hilo,US,80.26,clear sky,19.7297,-155.0900,
18,Hambantota,LK,76.55,overcast clouds,6.1241,81.1185,
30,Bengkulu,ID,75.67,moderate rain,-3.8004,102.2655,
31,Yenagoa,NG,76.73,scattered clouds,4.9247,6.2642,
33,Lazaro Cardenas,MX,84.74,overcast clouds,17.9583,-102.2000,
42,Borba,BR,85.19,overcast clouds,-4.3878,-59.5939,
43,Chuy,UY,80.55,clear sky,-33.6971,-53.4616,


In [16]:
headers = {"Connection": "keep-alive"}

In [17]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    print("Calling API")
    # 6e. Make request and retrieve the JSON data from the search. 
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotels = requests.get(base_url, params=params).json()
        print(hotels["results"][0]["name"])
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        print("inside!\n")
    except (KeyError, IndexError):
        print("Hotel not found... skipping.")
    except Exception as e:
        print(e)
hotel_df.head()    

Calling API
Sheraton Kauai Coconut Beach Resort
inside!

Calling API
Piaçabuçu News
inside!

Calling API
Bora-Bora Pearl Beach Resort & Spa
inside!

Calling API
Hilo Hawaiian Hotel
inside!

Calling API
Bungalow 63
inside!

Calling API
Grage Hotel Bengkulu
inside!

Calling API
Aridolf Resort Wellness and Spa
inside!

Calling API
quinta antigua
inside!

Calling API
Hotel da Amazônia
inside!

Calling API
Nuevo Hotel Plaza
inside!

Calling API
Pousada Porto Praia
inside!

Calling API
Fengshui Residence
inside!

Calling API
Paradise Inn
inside!

Calling API
Hotel Castelo
inside!

Calling API
Case de Passage De la Prefecture
inside!

Calling API
Villa Enata
inside!

Calling API
Taloa Heights
inside!

Calling API
Hotel Porto do Sol
inside!

Calling API
Beleza Pura B&B
inside!

Calling API
Hotel not found... skipping.
Calling API
Atlantis Hotel
inside!

Calling API
back to back rv park
inside!

Calling API
Scoop Guest House
inside!

Calling API
Isles Sunset Lodge
inside!

Calling API
Nuba Nile

Hotel Paternopoli
inside!

Calling API
Hotel not found... skipping.
Calling API
Villa Las Acacias
inside!

Calling API
Hotel Fórum
inside!

Calling API
Quality Suites Vila Velha
inside!

Calling API
Hotel Albergue de Olinda
inside!

Calling API
Austral Hotel
inside!

Calling API
Tarawa Boutique Hotel
inside!

Calling API
Pousada Porto De Canoas
inside!

Calling API
('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
Calling API
pondok pesantren mamba'ul 'ulum selo rejo
inside!

Calling API
Ricácio Hotel
inside!

Calling API
Nancy Montanes Bernardo
inside!

Calling API
Casa Morey Hotel & Restaurant
inside!

Calling API
Uruna Bay Retreat
inside!



,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Kapaa,US,80.58,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
11,Piacabucu,BR,78.55,scattered clouds,-10.4056,-36.4344,Piaçabuçu News
13,Faanui,PF,78.24,broken clouds,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
16,Hilo,US,80.26,clear sky,19.7297,-155.0900,Hilo Hawaiian Hotel
18,Hambantota,LK,76.55,overcast clouds,6.1241,81.1185,Bungalow 63


In [18]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City                   205
Country                205
Max Temp               205
Current Description    205
Lat                    205
Lng                    205
Hotel Name             205
dtype: int64

In [19]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [20]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [21]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))